# Simulation CV — oxydes de surface Au/Ni/Cu

Ce notebook détaille pas à pas le code de simulation de voltammétrie cyclique
pour la formation/réduction d'oxydes métalliques sur une électrode Au/Ni/Cu.

**Modèle** : réaction de surface (Langmuir), pas de diffusion en solution.  
**Résolution** : schéma implicite analytique (numpy), pas de FEM.  
**Réaction** : $M + H_2O \rightleftharpoons MOH + H^+ + e^-$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Constantes physiques

Toutes les grandeurs sont en **unités SI** (V, A, m², mol/m², s).  
Les potentiels sont exprimés **vs Ag/AgCl (KCl sat.)**.

In [ ]:
F = 96485.0        # C/mol  — constante de Faraday
R = 8.314          # J/(mol·K)
T = 298.15         # K (25 °C)
n_elec = 1         # électrons échangés par réaction
f = n_elec * F / (R * T)   # nF/RT ≈ 38.9 V⁻¹

A = 1.77e-6        # m² — surface de l'électrode (~1.5 mm de diamètre)
C_dl = 0.30        # F/m² — capacité de double couche (30 µF/cm²)

print(f"f = nF/RT = {f:.2f} V⁻¹")
print(f"RT/F = {R*T/F*1000:.1f} mV  (potentiel thermique)")

## 2. Paramètres des métaux (modèle multi-sites)

Chaque métal a :
- **E0_ox** : potentiel(s) d'oxydation — une *liste* de sites pour créer le plateau d'oxydation
- **E0_red** : potentiel de réduction — *unique* (pic cathodique sharp)
- **k0** : constante cinétique de surface [s⁻¹] (pas m/s !)
- **Γ_max** : densité maximale de sites [mol/m²]
- **α** : coefficient de transfert (symétrique = 0.5)

Le modèle **multi-sites** distribue E0_ox sur N sites uniformes pour reproduire
le plateau d'oxydation large observé expérimentalement sur l'or,
tout en gardant un pic de réduction unique et sharp.

Ici on prend **pH 0.3** (H₂SO₄ 0.5 M) comme exemple.

In [ ]:
# --- Au (or) : oxyde réversible, 20 sites entre 1.10 et 1.50 V ---
n_sites_Au = 20
E0_ox_Au = np.linspace(1.10, 1.50, n_sites_Au)   # plateau d'oxydation
frac_sites_Au = np.ones(n_sites_Au) / n_sites_Au  # poids uniformes
E0_red_Au = 0.90       # V — pic de réduction unique
k0_Au = 2.0            # s⁻¹
Gamma_Au = 4.0e-5      # mol/m²
alpha_Au = 0.5
frac_Au = 1.0          # fraction dans l'alliage

print(f"Au : {n_sites_Au} sites, E0_ox = [{E0_ox_Au[0]:.2f}, {E0_ox_Au[-1]:.2f}] V")
print(f"     E0_red = {E0_red_Au:.2f} V")
print(f"     Hystérésis ≈ {(np.mean(E0_ox_Au) - E0_red_Au)*1000:.0f} mV")

## 3. Potentiel triangulaire E(t)

Le potentiostat impose un balayage triangulaire :

$$E(t) = E_{start} + \nu \cdot t \cdot \text{direction}$$

avec $\nu$ = vitesse de balayage (0.1 V/s).  
La direction s'inverse aux bornes E_min / E_max ou lorsque
le courant dépasse un seuil (murs HER/OER).

In [ ]:
scan_rate = 0.1    # V/s
E_start = 0.0      # V
E_min = -0.35      # V (borne cathodique)
E_max = 1.65       # V (borne anodique)
dt = 1e-3          # s — pas de temps

def E_waveform(t, E_start, E_min, E_max, scan_rate):
    """Potentiel triangulaire : E_start → E_max → E_min → E_start."""
    t1 = (E_max - E_start) / scan_rate   # montée
    t2 = (E_max - E_min) / scan_rate     # descente
    t3 = (E_start - E_min) / scan_rate   # remontée
    t_cycle = t1 + t2 + t3
    t_mod = t % t_cycle
    if t_mod < t1:
        return E_start + scan_rate * t_mod
    elif t_mod < t1 + t2:
        return E_max - scan_rate * (t_mod - t1)
    else:
        return E_min + scan_rate * (t_mod - t1 - t2)

# Visualisation d'un cycle
t_demo = np.arange(0, 40, dt)
E_demo = np.array([E_waveform(ti, E_start, E_min, E_max, scan_rate) for ti in t_demo])

plt.plot(t_demo, E_demo)
plt.xlabel('t (s)'); plt.ylabel('E (V vs Ag/AgCl)')
plt.title('Potentiel triangulaire — 1 cycle')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Cinétique Butler-Volmer (multi-sites)

Pour chaque site d'oxydation $s$ de potentiel $E^0_{ox,s}$ :

$$k_{ox,s} = k_0 \exp\left(\alpha f (E - E^0_{ox,s})\right)$$

La réduction utilise un potentiel unique $E^0_{red}$ :

$$k_{red} = k_0 \exp\left(-(1-\alpha) f (E - E^0_{red})\right)$$

L'exponentielle est clampée à $\pm 20$ pour éviter les débordements numériques.

In [ ]:
def butler_volmer_rates(E, E0_ox, E0_red, k0, alpha):
    """Constantes de vitesse k_ox et k_red pour un site donné."""
    eta_ox = E - E0_ox
    eta_red = E - E0_red
    k_ox = k0 * np.exp(np.clip(alpha * f * eta_ox, -20, 20))
    k_red = k0 * np.exp(np.clip(-(1 - alpha) * f * eta_red, -20, 20))
    return k_ox, k_red

# Visualisation k_ox, k_red vs E pour Au (site moyen)
E_range = np.linspace(-0.5, 2.0, 500)
E0_ox_mean = np.mean(E0_ox_Au)
k_ox_demo = np.array([butler_volmer_rates(E, E0_ox_mean, E0_red_Au, k0_Au, alpha_Au)[0] for E in E_range])
k_red_demo = np.array([butler_volmer_rates(E, E0_ox_mean, E0_red_Au, k0_Au, alpha_Au)[1] for E in E_range])

plt.semilogy(E_range, k_ox_demo, 'r-', label=r'$k_{ox}$')
plt.semilogy(E_range, k_red_demo, 'b-', label=r'$k_{red}$')
plt.axvline(E0_ox_mean, color='r', ls=':', alpha=0.5, label=f'E0_ox = {E0_ox_mean:.2f} V')
plt.axvline(E0_red_Au, color='b', ls=':', alpha=0.5, label=f'E0_red = {E0_red_Au:.2f} V')
plt.xlabel('E (V)'); plt.ylabel('k (s⁻¹)')
plt.title('Constantes de vitesse Butler-Volmer — Au pH 0.3')
plt.legend(); plt.grid(True, alpha=0.3); plt.ylim(1e-6, 1e10)
plt.show()

## 5. Schéma implicite analytique pour dθ/dt

L'ODE de Langmuir pour la couverture de surface θ de chaque site :

$$\frac{d\theta}{dt} = k_{ox}(1-\theta) - k_{red} \cdot \theta$$

Cette ODE est **linéaire en θ**, ce qui permet un schéma implicite *exact* :

$$\theta^{n+1} = \frac{\theta^n + \Delta t \cdot k_{ox}}{1 + \Delta t (k_{ox} + k_{red})}$$

Ce schéma est **inconditionnellement stable** — pas besoin de solveur Newton,
pas de contrainte CFL. On clampe θ ∈ [0, 1] par sécurité.

In [ ]:
def implicit_step(theta_old, k_ox, k_red, dt):
    """Un pas de temps implicite analytique."""
    theta_new = (theta_old + dt * k_ox) / (1.0 + dt * (k_ox + k_red))
    return np.clip(theta_new, 0.0, 1.0)

## 6. Calcul du courant

Le courant faradique pour chaque site $s$ du métal $i$ :

$$I_{i,s} = x_i \cdot w_s \cdot n F A \, \Gamma_{max,i} \cdot \frac{d\theta_s}{dt}$$

où $x_i$ = fraction du métal dans l'alliage, $w_s$ = poids du site.

On y ajoute :
- **Courant capacitif** : $I_{cdl} = C_{dl} \cdot A \cdot \nu \cdot \text{direction}$
- **Mur HER** (cathodique) : $I_{HER} = -A \cdot i_{0,HER} \exp(-\alpha_{HER} f \eta_{HER})$
- **Mur OER** (anodique) : $I_{OER} = A \cdot i_{0,OER} \exp((1-\alpha_{OER}) f \eta_{OER})$

In [ ]:
# Paramètres HER/OER à pH 0.3
pH = 0.3
E0_HER = -0.10 - 0.059 * pH    # V — mur cathodique
E0_OER = 1.50 - 0.059 * pH     # V — mur anodique
i0_HER = 0.05     # A/m²
i0_OER = 0.15     # A/m²
alpha_HER = 0.5
alpha_OER = 0.5
I_threshold = 7e-6  # A — seuil d'inversion du balayage

print(f"pH {pH} : E_HER = {E0_HER:.3f} V, E_OER = {E0_OER:.3f} V")
print(f"Fenêtre d'eau = {E0_OER - E0_HER:.3f} V")

## 7. Simulation complète — Au pur, pH 0.3

La boucle temporelle intègre simultanément :
1. Le potentiel E(t) piloté dynamiquement (inversion au seuil de courant)
2. La couverture θ de chaque site (schéma implicite)
3. Le courant total (faradique + capacitif + HER/OER)

In [ ]:
# Allocation
n_cycles = 2
t_cycle = 2 * (E_max - E_min) / scan_rate
n_steps = int(n_cycles * t_cycle / dt)

t_arr = np.zeros(n_steps)
E_arr = np.zeros(n_steps)
I_arr = np.zeros(n_steps)
theta_arr = np.zeros((n_steps, n_sites_Au))  # θ par site

# État initial
E_current = E_start
scan_dir = +1         # +1 = anodique, -1 = cathodique
phase = 0             # 0: montée, 1: descente, 2: retour
cycles_done = 0
n_stab = 100          # pas de stabilisation (ignore transitoire)

for step in range(n_steps - 1):
    t_arr[step] = step * dt
    E_arr[step] = E_current
    E_next = E_current + scan_rate * dt * scan_dir

    I_step = 0.0

    # --- 1. Courant des oxydes (chaque site) ---
    for s in range(n_sites_Au):
        k_ox, k_red = butler_volmer_rates(
            E_next, E0_ox_Au[s], E0_red_Au, k0_Au, alpha_Au)
        theta_old = theta_arr[step, s]
        theta_new = implicit_step(theta_old, k_ox, k_red, dt)
        theta_arr[step + 1, s] = theta_new
        dtheta_dt = (theta_new - theta_old) / dt
        I_step += frac_Au * frac_sites_Au[s] * n_elec * F * A * Gamma_Au * dtheta_dt

    # --- 2. Mur HER ---
    eta_HER = E_current - E0_HER
    I_step += -A * i0_HER * np.exp(np.clip(-alpha_HER * f * eta_HER, -30, 30))

    # --- 3. Mur OER ---
    eta_OER = E_current - E0_OER
    I_step += A * i0_OER * np.exp(np.clip((1 - alpha_OER) * f * eta_OER, -30, 30))

    # --- 4. Courant capacitif ---
    I_step += C_dl * A * scan_rate * scan_dir

    I_arr[step] = I_step

    # --- 5. Pilotage du potentiel ---
    if step >= n_stab:
        if scan_dir == +1 and I_step > +I_threshold:
            scan_dir = -1; phase = 1
        elif scan_dir == -1 and I_step < -I_threshold:
            scan_dir = +1; phase = 2

    # Bornes de sécurité
    if scan_dir == +1 and E_current >= E_max:
        scan_dir = -1; phase = 1
    elif scan_dir == -1 and E_current <= E_min:
        scan_dir = +1; phase = 2

    # Détection fin de cycle
    if phase == 2 and scan_dir == +1 and E_current >= E_start:
        cycles_done += 1
        if cycles_done >= n_cycles:
            t_arr = t_arr[:step+2]; E_arr = E_arr[:step+2]
            I_arr = I_arr[:step+2]; theta_arr = theta_arr[:step+2]
            break
        phase = 0

    E_current += scan_rate * dt * scan_dir

print(f"Simulation terminée : {len(t_arr)} points, {cycles_done} cycles")
print(f"I_max = {np.max(I_arr)*1e6:.1f} µA,  I_min = {np.min(I_arr)*1e6:.1f} µA")

## 8. Voltammogramme cyclique I(E)

On trace la densité de courant $j$ (mA/cm²) en masquant les régions
où $|I|$ dépasse le seuil (murs HER/OER) et le transitoire initial.

In [ ]:
A_cm2 = A * 1e4  # m² → cm²
j = I_arr * 1e3 / A_cm2             # mA/cm²
j_threshold = I_threshold * 1e3 / A_cm2

mask = (np.abs(j) <= j_threshold)
mask[:n_stab] = False  # masquer le transitoire

fig, ax = plt.subplots(figsize=(10, 7))
for i in range(len(t_arr) - 1):
    if mask[i] and mask[i+1]:
        color = 'tab:red' if E_arr[i+1] > E_arr[i] else 'tab:blue'
        ax.plot(E_arr[i:i+2], j[i:i+2], color=color, lw=1.5)

ax.axhline(0, color='gray', ls='--', alpha=0.5)
ax.axhline(j_threshold, color='red', ls=':', alpha=0.4, label=f'Seuil ±{j_threshold:.1f} mA/cm²')
ax.axhline(-j_threshold, color='red', ls=':', alpha=0.4)
ax.set_xlabel('E (V vs Ag/AgCl)'); ax.set_ylabel('j (mA/cm²)')
ax.set_title(f'CV — Au pur, pH {pH}, ν = {scan_rate} V/s')
ax.set_xlim(E_arr[mask].min() - 0.05, E_arr[mask].max() + 0.05)
ax.set_ylim(-1.2 * j_threshold, 1.2 * j_threshold)
ax.legend(loc='upper left'); ax.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

## 9. Couverture de surface θ(E)

La couverture moyenne $\bar{\theta} = \sum_s w_s \theta_s$ montre
l'hystérésis caractéristique : l'oxydation se fait sur un plateau large
(multi-sites) tandis que la réduction est un pic sharp (E0_red unique).

In [ ]:
theta_mean = theta_arr @ frac_sites_Au  # moyenne pondérée des sites

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(E_arr, theta_mean, 'gold', lw=2)
ax1.set_xlabel('E (V)'); ax1.set_ylabel('θ')
ax1.set_title('Couverture θ(E)'); ax1.set_ylim(-0.05, 1.05)
ax1.grid(True, alpha=0.3)

ax2.plot(t_arr, theta_mean, 'gold', lw=2)
ax2.set_xlabel('t (s)'); ax2.set_ylabel('θ')
ax2.set_title('Couverture θ(t)'); ax2.set_ylim(-0.05, 1.05)
ax2.grid(True, alpha=0.3)

plt.tight_layout(); plt.show()

## 10. Ce que montre le voltammogramme

| Caractéristique | Origine physique |
|-----------------|------------------|
| **Plateau anodique large** | Multi-sites : 20 E0_ox distribués entre 1.10 et 1.50 V |
| **Pic cathodique sharp** | E0_red unique à 0.90 V — tous les oxydes se réduisent au même potentiel |
| **Hystérésis** | E0_ox ≠ E0_red — irréversibilité thermodynamique de l'oxyde |
| **Offset vertical** | Courant capacitif I_cdl = C_dl × A × ν |
| **Coupure aux bords** | Seuil de courant = murs HER/OER |

---

*Code source : `cv_surface_oxide.py` + `parameters_oxide.py`*  
*Dépendances : numpy, matplotlib — pas de FEM.*